# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns 
#sns.set(style="darkgrid")
plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False


from dateutil.relativedelta import relativedelta
from datetime import datetime as dt
import time

import warnings  
warnings.filterwarnings("ignore")

import sys
import pymysql
from sqlalchemy import create_engine
import json
import datetime 
from datetime import timedelta ,time 
import re

import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean

In [2]:

pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()

# 定义数据库连接

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入订单数据

In [4]:
sql1 = ''' -- 订单&风控信息  近10日数据   
SELECT date(om.create_time) as create_date,om.create_time,om.id as order_id ,om.order_number,om.all_money 
,om.status
,case om.`status` when  1 then "待支付" when  2 then "待发货" when  3 then "待收货" when  4 then "租赁中" when  5 then "待归还" 
when  6 then "待商家收货" when  7 then "退押中" when  8 then "已完成" when  10 then "已退款" when  11 then "待退押金" when  12 then "待审核" 
when  13 then "订单取消" when  15 then "检测中" when  9999 then "逾期订单" end as status2 
,case when locate('租物',pa.name)>0 or locate('租物',cc.name)>0 or locate('芝麻',pa.name)>0 or locate('芝麻',cc.name)>0  then '芝麻租物' when locate('抖音',pa.name)>0 then '抖音渠道' when locate('搜索',cc.name)>0 then '搜索渠道' else '其他渠道' end as channel_type 
,tod.sku_attributes,tod.product_name,tod.new_actual_money
,case when  locate('租完即送',tod.sku_attributes)>0 then '租完即送' else '租物归还' end as back_type
,om.user_mobile,tmu.true_name,tmu.id_card_num
,top.total_describes,tor.decision_result,om.cancel_reason
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.traceid') end,'"','') as trace_id 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.rejected') end,'"','') as rejected 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.result') end,'"','') as result 
,replace(case when JSON_VALID(tor.decision_result) THEN JSON_EXTRACT(tor.decision_result, '$.tips') end,'"','') as tips  
,cc.name as channel_name         -- 来源渠道
,pa.name as activity_name        -- 活动名称
,om.merchant_id,om.merchant_name
,topay.total_freeze_fund_amount 
,om.buy_service_product,tso.status as service_status 
,om.order_method, tor.update_time, tomt.reason
from  db_digua_business.t_order  om
left join db_digua_business.t_order_risk tor on om.id = tor.order_id
 -- 备注信息合并 
left join ( SELECT  t.order_id,JSON_ARRAYAGG(t.describes) as total_describes from db_digua_business.t_order_personnel t   GROUP BY 1 ) top 
on om.id = top.order_id 
-- 服务信息
left join  db_digua_business.t_service_order tso  on om.id = tso.order_id 
-- 渠道名称
left join db_digua_business.t_channel cc on om.channel = cc.scene 
-- 活动名称
left join db_digua_business.t_platform_activity pa on om.activity_id = pa.id
-- 用户信息 
left join db_digua_business.t_member_user tmu on om.user_id = tmu.id
-- 商品信息
left join db_digua_business.t_order_details tod on om.id = tod.order_id
-- 免押信息  
left join (SELECT t.*,row_number() over(partition by t.order_id order by t.pay_date desc) as rn 
from db_digua_business.t_order_pay t 
where t.pay_type = 'ZFBYSQ' and t.item_type=1 and t.`status` in (2,5) and t.trade_no is not null )  topay 
on topay.order_id=om.id   and  topay.rn = 1
-- 商家订单转移表
left join db_digua_business.t_order_merchant_transfer tomt on tomt.order_id=om.id
where om.user_mobile is not null 
and tmu.true_name not in ("刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文",'张娜','罗文龙','孔靖','彭康力','何薪华','夏玥','潘佳','包闻天','方全龙','李楠','向圆圆','黄兰娟','林婉婷','廖丽敏','李巧玲','李巧凤','刘三妹','蔡斯静'
,'陈宜诗','陈宝易','林寅钗','谢金凤','刘宏生','骆昌鑫','何静', '李珍珍')
-- and COALESCE(pa.name, '未知') not in ("1000单秘密计划","1000单秘密计划-无优惠","1000单曙光计划","线下门店3个月试行") 
-- AND tmer.shop_type !=2  -- 剔除商家
-- and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -1 Month )               -- 近15天数据
and  om.create_time >= DATE_ADD(CURRENT_DATE,INTERVAL -5 day )
;
'''

df_order = query(sql1)
df_order.shape

(82556, 32)

In [5]:
sql3 = '''
SELECT id_card, time, replace(case when JSON_VALID(data) THEN JSON_EXTRACT(data, '$.status') end,'"','') as status   FROM `db_credit`.risk_examine
'''
df_risk_examine = query(sql3)

# 合并数据

In [6]:
# df = df_order.merge(df_order_risk,left_on='订单id',right_on='order_id',how='left')
df = df_order.copy()
df.order_id.notnull().sum()

np.int64(82556)

# 处理日期

In [7]:

df["下单日期"]=df["create_time"].dt.date
df["下单日期"]=pd.to_datetime(df["下单日期"],errors="coerce")
df["下单日期"]
df["月份"]=df["下单日期"].dt.month
df['date']=df['create_time'].dt.date
df['hour']=df['create_time'].dt.hour
df.shape

(82556, 36)

In [8]:
df['拒绝理由'] = df["rejected"].str.replace("[","").str.replace("]","").str.replace('"','')
df['拒绝理由'].value_counts()

拒绝理由
                                                                               24511
命中蚁盾分3.0强拒                                                                     10838
命中冰鉴青云分强拒                                                                       9087
命中TD212强拒                                                                       6640
命中强拒加入临时黑名单                                                                     4530
                                                                               ...  
命中冰鉴青云分强拒, 命中极信sc32007分强拒, 命中自有模型 ascore_yl2311 强拒, 命中策略240703_10强拒                1
命中融安分低于700强拒, 评分等级低于C,命中拒绝, 命中模型2508组合1强拒, 命中模型2508组合2强拒                           1
命中冰鉴青云分强拒, 命中策略JDB002_212强拒, 命中JDB002_202098强拒, 命中模型2508组合1强拒, 命中景行分DE303强拒        1
命中冰鉴青云分强拒, 评分等级低于C,命中拒绝, 命中策略JDB002_212强拒, 命中模型2508组合3强拒                           1
命中自有模型 ascore2501_base 强拒, 评分等级低于D,命中拒绝                                            1
Name: count, Length: 536, dtype: int64

# 处理备注信息

In [9]:
df = df[df['sku_attributes'].notnull()]

In [10]:

df["取消原因"]=df["total_describes"].str.split("客户申请取消：").str[1].str.split("$").str[0].str.strip()
df["电审拒绝原因"]=df["total_describes"].str.split("审核不通过：").str[1].str.split("$").str[0].str.strip()
df["无法联系原因"]=df["total_describes"].str.split("用户无法联系：").str[1].str.split("$").str[0].str.strip()
df.loc[:,"商品类型"]=np.where(df["product_name"].str.contains('99新')|df["product_name"].str.contains('95新')|df["product_name"].str.contains('准新'),"二手","全新")  ##  准新
df.loc[:,"租赁方案"]=np.where(df["sku_attributes"].str.contains('租完即送'),"租完即送","租完归还")
df.loc[:,"押金类型"]=np.where(df["total_freeze_fund_amount"]>0,"部分免押","全免押")
df.loc[:,"优惠券使用否"]=np.where((df["new_actual_money"]-df["all_money"]>0)&(df["租赁方案"]=='租完即送'),"已使用","未使用")
df.loc[:,"是否进行预授权"]=np.where(df["total_freeze_fund_amount"].isnull(),"未预授权","已预授权")

def getcolor(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="颜色":
            return color_list[j]["value"]
df.loc[:,"颜色"]=df.apply(lambda x:getcolor(x["sku_attributes"]),axis=1)
df["颜色"].value_counts()  

# %%
def getneicun(s):
    color_list=json.loads(s)
    for j in range(0,len(color_list)):
        if color_list[j]["key"]=="内存":
            return color_list[j]["value"]
df.loc[:,"内存"]=df.apply(lambda x:getneicun(x["sku_attributes"]),axis=1)
df["内存"].value_counts() 

df.loc[:,"进件"]=np.where((df["status2"]=="待支付")|(df["status2"]=="订单取消"),"未进件","进件")
df["进件"].value_counts()

# %%
df.loc[:,"来源渠道"]=df["channel_name"].fillna("未知渠道")

In [11]:
df['免审'] = np.where(df.decision_result.str.contains(pat='免人审',regex=False),1,0)
df['免审'].value_counts()

免审
0    80070
1     2486
Name: count, dtype: int64

# 渠道归属

In [12]:
def qudao_type(a,b,c):
    a = str(a)
    b = str(b)
    if "租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "抖音" in b:
        return "抖音渠道"
    elif "搜索" in a:
        return "搜索渠道"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"
    elif c == 1:
        return "芝麻租物"   
    elif "叮咚直播" in a:
        return "叮咚直播"
    elif "租瓜直播2号" in a:
        return "租瓜直播2号"
    elif "租瓜直播" in a:
        return "租瓜直播"
    elif "直播" in a:
        return "支付宝直播"
    elif "繁星" in a:
        return "繁星"
    elif "生活号" in a:
        return "生活号"
    elif "群" in a:
        return "支付宝社群"
    elif "付费灯火" in a:
        return "付费灯火"
    else :
        return a
    

# %%
df.loc[:,"归属渠道"]=df.apply(lambda x:qudao_type(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df["归属渠道"].value_counts()

归属渠道
芝麻租物                              50107
搜索渠道                              14353
未知渠道                               9034
单人聊天会话中的小程序消息卡片（分享）                2854
八派信息                               1290
支付宝客户端首页                            979
小程序商家消息（服务提醒）                       730
京享租右卡                               591
抖音渠道                                498
生活号                                 353
支付宝直播                               346
我的小程序入口                             298
其他渠道场景渠道。                           241
派金花                                 238
支付宝社群                               159
订阅消息                                149
宜品花                                 118
付费流量(通过商家数字推广平台，灯火等投放的广告)           111
苹果BD21                               37
曙光计划                                 19
九州信息                                 18
扫描二维码                                15
系统桌面图标                                4
赛租机                                   3
收藏卡片01                             

# 订单去重

In [13]:
dict_status_code = {
    "订单取消": 1,
    "待支付": 2,
    "已退款": 3, 
    "待审核": 4,
    "待发货": 5,
    "待收货": 6,
    "租赁中": 7,
    "已完成": 8
}
df["状态编码"]=df["status2"].map(dict_status_code)


df.sort_values(by=["下单日期","状态编码"],inplace=True)


#删除重复单号
df.drop_duplicates(subset=["order_id"],inplace=True)
#删除身份证空值行
df.dropna(subset=["id_card_num"],axis=0,inplace=True)
#去刷单订单
df.drop(df[df['total_describes'].str.contains(pat='panli',regex=False)==True].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单秘密计划-无优惠"].index,inplace=True)
df.drop(df[df['activity_name']=="1000单曙光计划"].index,inplace=True)
df.drop(df[df['activity_name']=="线下门店3个月试行"].index,inplace=True)
#删除身份证空值行
df.dropna(subset=["status2"],axis=0,inplace=True)


#删除重复订单
df.drop_duplicates(subset=["order_id"],inplace=True)
df.drop_duplicates(subset=["true_name","user_mobile","id_card_num","下单日期"],keep="last",inplace=True)


df.drop(df[df['true_name'].isin(["刘鹏","谢仕程","潘立","洪柳","陈锦奇","周杰","卢腾标","孔靖","黄娟","钟福荣","邱锐杰","唐林华"
,"邓媛斤","黄子南","刘莎莎","赖瑞彤","孙子文", '李珍珍'])].index,inplace=True)

# 定义状态

In [14]:
# 判断 前置拦截   机审强拒   出库前风控强拒
df['是否前置拦截'] = np.where((df.result.str.contains('id_card不得为空') )|(df.result.str.contains('mobile校验不通过')) |(df.result.str.contains('name校验不通过'))
                        |(df.result.str.contains('年龄超过49岁或低于18岁'))|(df.result=='风控未通过')|(df.result=='黑名单用户'),1,0)
df['是否机审强拒'] = np.where((df.result.str.contains('风控拒绝') & (~df.result.str.contains('命中出库前风控流强拒').fillna(False)) ),1,0)
df['是否出库前风控强拒'] = np.where((df.result.str.contains('命中出库前风控流强拒').fillna(False)) |((df.total_describes.str.contains('蚂蚁数控风险等级').fillna(False)) & (~df.result.str.contains('黑名单用户').fillna(False))) ,1,0)


In [15]:

df.loc[:,"审核状态"]=df.apply(lambda x: clean.reject_type(x["拒绝理由"],x["进件"],x["电审拒绝原因"],x["取消原因"],x["status2"],x["无法联系原因"],x["total_describes"],x['是否前置拦截'],x['是否机审强拒'],x['是否出库前风控强拒']),axis=1)#,x["无法联系原因"]
df["审核状态"].value_counts()  

审核状态
进件前取消      82909
前置拦截       66743
机审强拒       46382
人审拒绝       11065
出库          7201
已退款         3545
客户取消        2021
出库前风控强拒     1642
无法联系        1577
待审核          242
待归还           11
待商家收货          7
检测中            2
Name: count, dtype: int64

# 保留商家的数据

In [16]:
df_contain = df.copy()

# 剔除商家数据

In [17]:
#剔除商家数据只保留自营租机业务数据
df = clean.drop_merchant(df)

# 去重订单明细导出

In [18]:
from datetime import datetime  
today = datetime.today() 
t_date = today.date().strftime('%Y-%m-%d')
t_date

'2025-07-31'

# 导入服务订单

In [19]:
df["下单时段"]=df["create_time"].astype(str).str[:14]

# %%
df_j = df[df["进件"]=="进件"]
df_j.shape

(71859, 57)

In [20]:
df["待审核"] = np.where(df["审核状态"]=='待审核',1,0)
df["前置拦截"] = np.where(df["审核状态"]=='前置拦截',1,0)
df["机审强拒"] = np.where(df["审核状态"]=='机审强拒',1,0)
df["人审拒绝"] = np.where(df["审核状态"]=='人审拒绝',1,0)
df["客户取消"] = np.where(df["审核状态"]=='客户取消',1,0)
df["出库前风控强拒"] = np.where(df["审核状态"]=='出库前风控强拒',1,0)
df["无法联系"] = np.where(df["审核状态"]=='无法联系',1,0)
df["是否进件"] = np.where(df["进件"]=='进件',1,0)
df["是否出库"] = np.where(df["status"].isin([2,3,4,5,6,8,15]),1,0)

df["进件前取消"] = np.where(df["审核状态"]=='进件前取消',1,0)
df['机审通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0), 1, 0)
df['风控通过件'] = np.where((df['是否进件']==1)&(df['机审强拒']==0)&(df['人审拒绝']==0), 1, 0)

df['是否出库'] = np.where((df['人审拒绝']==0)&(df['客户取消']==0)&(df['无法联系']==0)&(df['待审核']==0)&(df['是否出库']==1), 1, 0)


In [21]:
#读取导入出库台账数据
f_path_ck = "F:/myfile/p站数据/台账数据/维客壹佰2023&2024年台账.xlsx"
# wb = pd.ExcelFile(f_path_ck)  
# df_ck = pd.read_excel(wb, sheet_name="2023")
df_ck = pd.read_excel(f_path_ck,sheet_name="2023")   #,skiprows=0 header 
df_ck.shape

(49985, 19)

In [22]:
df_ck['溢价费订单'] = "A"+df_ck[df_ck['备注'].str.contains(pat='溢价费',regex=False)==True]['备注'].str.extract('(\d+.\d+)')
df_yijia = df_ck[df_ck['溢价费订单'].str.len()>=16][['溢价费订单',"已付金额"]]
df_yijia=df_yijia.rename(columns={'溢价费订单':"单号","已付金额":"溢价费"})
df_yijia_ck = pd.merge(df_ck,df_yijia,left_on="订单号", right_on="单号",how="left")
dfck = pd.merge(df_yijia_ck,df,left_on="订单号", right_on="order_number")

dfck.drop_duplicates(subset=["订单号"],inplace=True)
dfck.shape#.to_excel(f"C:/Users/Administrator/Desktop/报表模板/出库台账最终整理/11月出库补充1220.xlsx")

(2549, 91)

In [23]:
dfck[dfck["status2"]=="已退款"]

dfck.drop(dfck[dfck["status2"]=="已退款"].index,inplace=True)

dfck[dfck["status2"]=="已退款"]

# 删除 露营设备 出库 
dfck.drop(dfck[dfck["类目"]=="露营设备"].index,inplace=True)


# iPhone 16 全系列

In [24]:
qd_names = ['归属渠道', '来源渠道']
qd_type = ['去重订单数', '是否进件']
indx = 0
df2 = df.copy()
df2 = clean.drop_rejected_merchant(df2)
df2['date']=df2['create_time'].dt.date
df2['hour']=df2['create_time'].dt.hour

# iPhone 华为


In [25]:
# df_fsh = df[~df['product_name'].str.contains(r'iPhone|华为|vivo|一加|iQOO|OPPO|小米|Redmi|天语|红米|真我|荣耀|畅享|Mate|nova|Xiaomi|测试商品|IPhone')]
# df_pb = df_fsh[df_fsh['product_name'].str.contains(r'平板|台式机|电脑配件|笔记本|商务本|游戏本')]
# df_sb = df_fsh[df_fsh['product_name'].str.contains(r'手表')]
# df_xj = df_fsh[df_fsh['product_name'].str.contains(r'相机|镜头|微单|手持云台|单反|摄影配件|拍立得|摄影机|影棚器材|镜头|无人机')]
# df_hw = df_fsh[df_fsh['product_name'].str.contains(r'探路者|帐篷|桌子|椅子|天幕|户外套装|随身wifi|望远镜|翻译机|户外装备|户外照明')]
# df_rtt = df_fsh[df_fsh['product_name'].str.contains(r'任天堂|游戏卡|游戏机')]
# df_yq = df_fsh[df_fsh['product_name'].str.contains(r'耳机|音响|音箱')]
# # df['product_name'].value_counts().head(50)
# # df[df['product_name'].str.contains(r'iPad')]['product_name']
# df_pb['product_name'].drop_duplicates()
# df_pb['order_number'].count()
# df_sb['order_number'].count()
# df_xj['order_number'].count()
# df_hw['order_number'].count()
# df_rtt['order_number'].count()
# df['order_number'].count()
# df_fsj_all = pd.DataFrame({
#     '平板|电脑' : [df_pb['order_number'].count()],
#     '手表': [df_sb['order_number'].count()],
#     '相机/摄影': [df_xj['order_number'].count()],
#     '户外': [df_hw['order_number'].count()],
#     '游戏设备': [df_rtt['order_number'].count()],
#     '乐器影音': [df_yq['order_number'].count()],
#     '非手机汇总': [df_fsh['order_number'].count()],
#     '总体汇总': [df['order_number'].count()]
# })
# df_fsj_all['平板|电脑比例'] = df_fsj_all['平板|电脑'] / df_fsj_all['总体汇总'] * 100
# df_fsj_all['平板|电脑比例'] = df_fsj_all['平板|电脑比例'].apply(lambda x: f'{x:.2f}%')

# df_fsj_all['手表比例'] = df_fsj_all['手表'] / df_fsj_all['总体汇总'] * 100
# df_fsj_all['手表比例'] = df_fsj_all['手表比例'].apply(lambda x: f'{x:.2f}%')

# df_fsj_all['相机/摄影比例'] = df_fsj_all['相机/摄影'] / df_fsj_all['总体汇总'] * 100
# df_fsj_all['相机/摄影比例'] = df_fsj_all['相机/摄影比例'].apply(lambda x: f'{x:.2f}%')

# df_fsj_all['户外比例'] = df_fsj_all['户外'] / df_fsj_all['总体汇总'] * 100
# df_fsj_all['户外比例'] = df_fsj_all['户外比例'].apply(lambda x: f'{x:.2f}%')

# df_fsj_all['游戏设备比例'] = df_fsj_all['游戏设备'] / df_fsj_all['总体汇总'] * 100
# df_fsj_all['游戏设备比例'] = df_fsj_all['游戏设备比例'].apply(lambda x: f'{x:.2f}%')

# df_fsj_all['乐器影音比例'] = df_fsj_all['乐器影音'] / df_fsj_all['总体汇总'] * 100
# df_fsj_all['乐器影音比例'] = df_fsj_all['乐器影音比例'].apply(lambda x: f'{x:.2f}%')

# df_fsj_all['非手机比例'] = df_fsj_all['非手机汇总'] / df_fsj_all['总体汇总'] * 100
# df_fsj_all['非手机比例'] = df_fsj_all['非手机比例'].apply(lambda x: f'{x:.2f}%')

# df_fsj_all


In [26]:
def HuaWei(df, date1, date2):
    df = df[df['product_name'].str.contains(r'华为')]
    # 定义更完善的正则表达式模式
    pattern = re.compile(r'(华为 ?(Mate|Pura|畅享|P|nova) ?\d+ ?(Pro\+|Pro|Plus\+|Plus|Flip\+|Flip)?|nova ?\d+ ?(Pro\+|Pro|Ultra\+|Ultra|Flip\+|Flip)?|华为 ?nova ? Flip?)')
    # 使用 apply 和 lambda 函数匹配描述中的手机型号
    df['型号'] = df['product_name'].apply(lambda x: pattern.findall(x))
    # 将匹配结果展平为单个字符串
    df['型号'] = df['型号'].apply(lambda x: [item[0] for item in x])
    # 过滤出包含手机型号的行
    df_filtered = df[df['型号'].apply(lambda x: len(x) > 0)]
    # 展开列表，确保每个型号都在单独的一行中
    df_expanded = df_filtered.explode('型号')
    df_expanded['型号'] = df_expanded['型号'].str.replace(' ', '')

    df_expanded['型号内存'] = df_expanded['型号']+'_'+df_expanded['内存']
    df_expanded = df_expanded[(df_expanded['下单日期']>=date1)&(df['下单日期']<=date2)]

    return df_expanded



In [27]:
def iPhone(df, model, date1, date2):
    df = df[df['product_name'].str.contains(fr'iPhone {model}|iPhone{model}')]
    df = df[df['product_name']!='iPhone 16 2024年新机 200元权益包']
    df['product_name'] = df['product_name'].str.replace(' ', '')
    df['型号'] = df['product_name'].str.extract(fr'(iPhone ?{model}(ProMax|Pro|Plus)?)', expand=True)[0]
    df['型号'] = df['型号'].str.replace(' ', '')
    df = df[df['型号'].notna()]
    df['型号内存'] = df['型号']+'_'+df['内存']
    df = df[(df['下单日期']>=date1)&(df['下单日期']<=date2)]
    # df = df[df['create_time'].dt.hour]
    # df_15 = df_15[df_15['下单日期']=='2024-10-01']

    return df
    # df['型号'].drop_duplicates()

In [38]:
def iPhone_model(df, df1, df_risk_examine, model):

    if type(model)==str:
        df_group1 = df.groupby(model).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum'})
        df_group2 = df1.groupby(model).agg({'机审强拒':'sum','机审通过件':'sum','人审拒绝':'sum','风控通过件':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
        df_group3 = pd.merge(df_group1, df_group2, on=model, how='inner')
        # 拼接下蚂蚁租机的被拒数据
        df_xmy = all_models.xmy(df, df_risk_examine, model)
        df_group = pd.merge(df_xmy, df_group3, on=model, how='right')
        df_group = df_group.fillna(0)
        df_group['机审强拒'] = df_group['机审强拒'] + df_group['小蚂蚁机审强拒']
        df_group['机审通过件'] = df_group['机审通过件'] + df_group['小蚂蚁人审拒绝'] + df_group['order_risk_ex']  # +除机审强拒的单
        df_group['风控通过件'] = df_group['风控通过件'] + df_group['order_risk_ex']  # +出库前强拒的单
        df_group['人审拒绝'] = df_group['人审拒绝'] + df_group['小蚂蚁人审拒绝']
        df_group['出库前风控强拒'] = df_group['出库前风控强拒'] + df_group['order_risk_ex']
        df_group.rename(columns={'order_id':'去重订单数', '是否进件': '进件', '是否出库': '出库'},inplace=True )
        # +过滤掉18点后的数据
        df_group = df_group.query('hour<18')
        df_group = df_group.groupby([f'{model}'])[['去重订单数','进件','前置拦截','机审强拒','人审拒绝','客户取消','无法联系','出库前风控强拒','待审核','出库']].cumsum()
        df_group = df_group.groupby([f'{model}'])[['去重订单数','进件','前置拦截','机审强拒','人审拒绝','客户取消','无法联系','出库前风控强拒','待审核','出库']].last()
    else:
        df_group1 = df.groupby(model).agg({'order_id':'size','是否进件':'sum','前置拦截':'sum'})
        df_group2 = df.groupby(model).agg({'机审强拒':'sum','机审通过件':'sum','人审拒绝':'sum','风控通过件':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
        df_group3 = pd.merge(df_group1, df_group2, on=model, how='inner')
        # 拼接下蚂蚁租机的被拒数据
        df_xmy = all_models.xmy(df, df_risk_examine, model)
        df_group = pd.merge(df_xmy, df_group3, on=model, how='right')
        df_group = df_group.fillna(0)
        df_group['机审强拒'] = df_group['机审强拒'] + df_group['小蚂蚁机审强拒']
        df_group['机审通过件'] = df_group['机审通过件'] + df_group['小蚂蚁人审拒绝'] + df_group['order_risk_ex']  # +除机审强拒的单
        df_group['风控通过件'] = df_group['风控通过件'] + df_group['order_risk_ex']  # +出库前强拒的单
        df_group['人审拒绝'] = df_group['人审拒绝'] + df_group['小蚂蚁人审拒绝']
        df_group['出库前风控强拒'] = df_group['出库前风控强拒'] + df_group['order_risk_ex']
        df_group.rename(columns={'order_id':'去重订单数', '是否进件': '进件', '是否出库': '出库'},inplace=True )
    
    
    df_group['进件占比'] = df_group['进件']/df_group['去重订单数'] * 100
    df_group['进件占比'] = df_group['进件占比'].apply(lambda x: f'{x:.2f}%')

    df_group['出库占比'] = df_group['出库']/df_group['去重订单数'] * 100
    df_group['出库占比'] = df_group['出库占比'].apply(lambda x: f'{x:.2f}%')

    df_group['进件出库率'] = df_group['出库']/df_group['进件'] * 100
    df_group['进件出库率'] = df_group['进件出库率'].apply(lambda x: f'{x:.2f}%')

    df_group = df_group[['去重订单数', '进件', '进件占比', '出库', '出库占比', '进件出库率']]
    return df_group


In [29]:
def group_data(df0, df1, df_risk_examine, model):
    df0['create_time'] = pd.to_datetime(df0['create_time'])
    # 策略241205
    # df_241205 = df0[df0.tips.str.contains('策略241205', regex=False)==True]
    # df_241205_group = df_241205.groupby(model).agg({'机审通过件': 'sum'})
    # df_241205_group.rename(columns={'机审通过件': '机审通过件_241205'}, inplace=True)
    
    df_group1 = df0.groupby(model).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum'})
    df_group2 = df1.groupby(model).agg({'机审强拒':'sum','机审通过件':'sum','人审拒绝':'sum','风控通过件':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_group3 = pd.merge(df_group1, df_group2, on=model, how='inner')
    # 拼接下蚂蚁租机的被拒数据
    df_xmy = all_models.xmy(df0, df_risk_examine, model)
    df_group = df_group3.merge(df_xmy, on=model, how='left')#.merge(df_241205_group, on=model, how='left')
    df_group = df_group.fillna(0)
    df_group['机审强拒'] = df_group['机审强拒'] + df_group['小蚂蚁机审强拒'] #+ df_group['机审通过件_241205']
    df_group['机审通过件'] = df_group['机审通过件'] + df_group['小蚂蚁人审拒绝'] + df_group['order_risk_ex']  # +除机审强拒的单
    df_group['风控通过件'] = df_group['风控通过件'] + df_group['order_risk_ex']  # +出库前强拒的单
    df_group['人审拒绝'] = df_group['人审拒绝'] + df_group['小蚂蚁人审拒绝']
    df_group['出库前风控强拒'] = df_group['出库前风控强拒'] + df_group['order_risk_ex']
    df_group.rename(columns={'order_id':'去重订单数'},inplace=True )

    df_group = df_group.query('hour<=23')

    df_group = df_group.groupby([f'{model[0]}'])[['去重订单数','是否进件','进件前取消','前置拦截','机审强拒','机审通过件','人审拒绝','风控通过件','客户取消','无法联系','出库前风控强拒','待审核','是否出库']].cumsum()
    df_group = df_group.groupby([f'{model[0]}'])[['去重订单数','是否进件','进件前取消','前置拦截','机审强拒','机审通过件','人审拒绝','风控通过件','客户取消','无法联系','出库前风控强拒','待审核','是否出库']].last()

    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["是否进件"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group["是否进件"]
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    # df_group["机审通过件"] = df_group["是否进件"]-df_group["机审强拒"]

    # df_group["风控通过件"] = df_group["是否进件"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group["是否进件"]
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group["是否进件"]
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group["是否进件"]
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group["是否进件"]
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group["是否进件"]
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["是否出库"]/df_group["是否进件"]
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["是否出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))

    return df_group

In [30]:
def group_data_new(df0, df1, df_risk_examine, model):
    df0['create_time'] = pd.to_datetime(df0['create_time'])
# 策略241205
#     df_241205 = df0[df0.tips.str.contains('策略241205', regex=False)==True]
#     df_241205_group = df_241205.groupby(model).agg({'机审通过件': 'sum'})
#     df_241205_group.rename(columns={'机审通过件': '机审通过件_241205'}, inplace=True)
    
    df_group1 = df0.groupby(model).agg({'order_id':'size','是否进件':'sum','进件前取消':'sum','前置拦截':'sum'})
    df_group2 = df1.groupby(model).agg({'机审强拒':'sum','机审通过件':'sum','人审拒绝':'sum','风控通过件':'sum','客户取消':'sum','无法联系':'sum', '出库前风控强拒':'sum','待审核':'sum','是否出库':'sum'})
    df_group3 = pd.merge(df_group1, df_group2, on=model, how='inner')
    # 拼接下蚂蚁租机的被拒数据
    df_xmy = all_models.xmy(df0, df_risk_examine, model)
    df_group = df_group3.merge(df_xmy, on=model, how='left')#.merge(df_241205_group, on=model, how='left')
    df_group = df_group.fillna(0)
    df_group['机审强拒'] = df_group['机审强拒'] + df_group['小蚂蚁机审强拒'] #+ df_group['机审通过件_241205']
    df_group['机审通过件'] = df_group['机审通过件'] + df_group['小蚂蚁人审拒绝'] + df_group['order_risk_ex']  # +除机审强拒的单
    df_group['风控通过件'] = df_group['风控通过件'] + df_group['order_risk_ex']  # +出库前强拒的单
    df_group['人审拒绝'] = df_group['人审拒绝'] + df_group['小蚂蚁人审拒绝']
    df_group['出库前风控强拒'] = df_group['出库前风控强拒'] + df_group['order_risk_ex']
    df_group.rename(columns={'order_id':'去重订单数'},inplace=True )

    df_group = df_group.query('hour<=23')

    df_group = df_group.groupby(model[:2])[['去重订单数','是否进件','进件前取消','前置拦截','机审强拒','机审通过件','人审拒绝','风控通过件','客户取消','无法联系','出库前风控强拒','待审核','是否出库']].cumsum()
    df_group = df_group.groupby(model[:2])[['去重订单数','是否进件','进件前取消','前置拦截','机审强拒','机审通过件','人审拒绝','风控通过件','客户取消','无法联系','出库前风控强拒','待审核','是否出库']].last()


    df_group["拦截率"] = df_group["前置拦截"]/df_group["去重订单数"]
    df_group["拦截率"] = df_group["拦截率"].apply(lambda x:format(x,".2%"))

    df_group["进件前取消率"] = df_group["进件前取消"]/df_group["去重订单数"]
    df_group["进件前取消率"] = df_group["进件前取消率"].apply(lambda x:format(x,".2%"))

    df_group['预授权通过率'] = df_group["是否进件"]/df_group["去重订单数"]
    df_group["预授权通过率"] = df_group["预授权通过率"].apply(lambda x:format(x,".2%"))

    df_group["强拒比例"] = df_group["机审强拒"]/df_group["是否进件"]
    df_group["强拒比例"] = df_group["强拒比例"].apply(lambda x:format(x,".2%"))

    # df_group["机审通过件"] = df_group["是否进件"]-df_group["机审强拒"]

    # df_group["风控通过件"] = df_group["是否进件"]-df_group["机审强拒"]-df_group["人审拒绝"]
    df_group["风控通过率"] = df_group["风控通过件"]/df_group["是否进件"]
    df_group["风控通过率"] = df_group["风控通过率"].apply(lambda x:format(x,".2%"))

    df_group["取消率"] = df_group["客户取消"]/df_group["是否进件"]
    df_group["取消率"] = df_group["取消率"].apply(lambda x:format(x,".2%"))

    df_group["无法联系占比"]=df_group["无法联系"]/df_group["是否进件"]
    df_group["无法联系占比"]=df_group["无法联系占比"].apply(lambda x:format(x,".2%"))

    df_group["人审拒绝率"]=df_group["人审拒绝"]/df_group["是否进件"]
    df_group["人审拒绝率"]=df_group["人审拒绝率"].apply(lambda x:format(x,".2%"))

    df_group["出库前强拒比例"]=df_group["出库前风控强拒"]/df_group["是否进件"]
    df_group["出库前强拒比例"]=df_group["出库前强拒比例"].apply(lambda x:format(x,".2%"))

    df_group["进件出库率"]=df_group["是否出库"]/df_group["是否进件"]
    df_group["进件出库率"]=df_group["进件出库率"].apply(lambda x:format(x,".2%"))

    df_group["订单出库率"]=df_group["是否出库"]/df_group["去重订单数"]
    df_group["订单出库率"]=df_group["订单出库率"].apply(lambda x:format(x,".2%"))
    return df_group

## 设置时间

In [45]:

date1 = '2025-07-30'
date2 = '2024-07-31'
month_date1 = date1.split('-')[1]
day_date1 = date1.split('-')[2]
month_date2 = date2.split('-')[1]
day_date2 = date2.split('-')[2]
df_zm = df[df['归属渠道']=='芝麻租物']
df_zm2 = df2[df2['归属渠道']=='芝麻租物']

In [46]:
df_zm_jj = df_zm[df_zm['是否进件']==1]
# df_zm_jj[~df_zm_jj['product_name'].str.contains(r'iPhone|华为|手表|电脑|野餐垫|充电宝')]['product_name'].drop_duplicates()
# df_zm_all = iPhone_model(df_zm_jj, '下单日期')
# df_zm_all
# with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/租物进件占比_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl') as writer:
#     df_zm_all.to_excel(writer, sheet_name='租物进件占比')

## iPhone系列

### 进件出库占比

In [47]:
# 总体 iPhone15
df_15 = iPhone(df, 15, date1, date2)
df_152 = iPhone(df2, 15, date1, date2)
df_group_15 = iPhone_model(df_15,df_152,df_risk_examine, ['型号', 'hour'])
df_group_15_model = iPhone_model(df_15,df_152,df_risk_examine, ['型号内存', 'hour'])
# 芝麻租物 iPhone15
df_15_zm = iPhone(df_zm, 15, date1, date2)
df_15_zm2 = iPhone(df_zm2, 15, date1, date2)
df_group_15_zm = iPhone_model(df_15_zm,df_15_zm2,df_risk_examine, ['型号', 'hour'])
df_group_15_model_zm = iPhone_model(df_15_zm,df_15_zm2,df_risk_examine, ['型号内存', 'hour'])

TypeError: Can only merge Series or DataFrame objects, a <class 'tuple'> was passed

In [44]:
# 总体 iPhone16
df_16 = iPhone(df, 16, date1, date2)
df_162 = iPhone(df2, 16, date1, date2)
df_group_16 = iPhone_model(df_16,df_162,df_risk_examine, ['型号', 'hour'])
df_group_16_model = iPhone_model(df_16,df_162,df_risk_examine, ['型号内存', 'hour'])

# 芝麻租物 iPhone16
df_16_zm = iPhone(df_zm, 16, date1, date2)
df_16_zm2 = iPhone(df_zm2, 16, date1, date2)
df_group_16_zm = iPhone_model(df_16_zm,df_16_zm2,df_risk_examine, ['型号', 'hour'])
df_group_16_model_zm = iPhone_model(df_16_zm,df_16_zm2,df_risk_examine, ['型号内存', 'hour'])

TypeError: Can only merge Series or DataFrame objects, a <class 'tuple'> was passed

In [ ]:
# 总体
df_group_iPhone_model = pd.concat([df_group_15, df_group_16])
df_group_model = pd.concat([df_group_15_model, df_group_16_model])

# 芝麻租物
df_group_iPhone_model_zm = pd.concat([df_group_15_zm, df_group_16_zm])
df_group_model_zm  = pd.concat([df_group_15_model_zm, df_group_16_model_zm])

In [ ]:
Today1 = datetime.now().strftime('%Y%m%d')
# with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/iPhone机型转化/机型转化_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl') as writer:
#     df_group_iPhone_model.to_excel(writer, sheet_name='iPhone机型转化（汇总）')
# with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/iPhone机型转化/机型转化_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl', mode='a') as writer:
#     df_group_15.to_excel(writer, sheet_name='iPhone15')
#     df_group_16.to_excel(writer, sheet_name='iPhone16')
#     df_group_model.to_excel(writer, sheet_name='iPhone机型转化(分内存)')
#     df_group_15_model.to_excel(writer, sheet_name='iPhone15(分内存)')
#     df_group_16_model.to_excel(writer, sheet_name='iPhone16(分内存)')
#     df_group_15_model_zm.to_excel(writer, sheet_name='iPhone15(分内存_芝麻租物)')
#     df_group_16_model_zm.to_excel(writer, sheet_name='iPhone16(分内存_芝麻租物)')

### 总体转化

In [ ]:
df_all = df[(df['下单日期']>=date1)&(df['下单日期']<=date2)]
df_all2 = df2[(df2['下单日期']>=date1)&(df2['下单日期']<=date2)]
df_all_group = group_data(df_all,df_all2, df_risk_examine, ['下单日期', 'hour'])
df_all_group = df_all_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝",  "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_all_group

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,,,
2024-11-14,2717,715,26.32%,529,19.47%,65.0,12.29%,175.0,17.0,158.0,29.87%,39,21.0,3.97%,84,15.88%,3.09%
2024-11-15,3043,783,25.73%,617,20.28%,121.0,19.61%,174.0,22.0,152.0,24.64%,49,14.0,2.27%,76,12.32%,2.50%


### iPhone 转化数据

In [ ]:
df_15_all = group_data(df_15,df_152,df_risk_examine, ['型号', 'hour'])
# df_15_all = group_data(df_15, '下单日期')
# df_15_all = df_15_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", '人审拒绝', "风控通过件","风控通过率","进件前取消", "出库前风控强拒","出库前强拒比例","是否出库",'取消率','人审拒绝率',"进件出库率","订单出库率"]]


df_15_all = df_15_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_15_all

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
型号,,,,,,,,,,,,,,,,
iPhone15,223,57,25.56%,46,20.63%,0.0,0.00%,20.0,19.0,41.30%,10,1.0,2.17%,5,10.87%,2.24%
iPhone15Plus,4,0,0.00%,2,50.00%,0.0,0.00%,2.0,2.0,100.00%,0,0.0,0.00%,2,100.00%,50.00%
iPhone15Pro,154,50,32.47%,24,15.58%,0.0,0.00%,11.0,10.0,41.67%,3,0.0,0.00%,4,16.67%,2.60%
iPhone15ProMax,931,230,24.70%,181,19.44%,0.0,0.00%,42.0,36.0,19.89%,6,5.0,2.76%,22,12.15%,2.36%


In [ ]:
df_15_date_group = group_data_new(df_15,df_152,df_risk_examine, ['型号', 'date', 'hour'])[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", '人审拒绝', '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_15_date_group
# with pd.ExcelWriter('F:/需求/瑞彤需求/iPhone转化.xlsx', engine='openpyxl') as writer:
#     df_15_date_group.to_excel(writer, sheet_name='iPhone15')

去重订单数  前置拦截     拦截率  是否进件   预授权通过率  机审强拒   强拒比例  \
型号             date                                                          
iPhone15       2024-11-14     83    17  20.48%    19   22.89%   0.0  0.00%   
               2024-11-15     76    25  32.89%    17   22.37%   0.0  0.00%   
iPhone15Plus   2024-11-15      2     0   0.00%     2  100.00%   0.0  0.00%   
iPhone15Pro    2024-11-14     48    17  35.42%     9   18.75%   0.0  0.00%   
               2024-11-15     50    15  30.00%    10   20.00%   0.0  0.00%   
iPhone15ProMax 2024-11-14    295    69  23.39%    51   17.29%   0.0  0.00%   
               2024-11-15    481   113  23.49%   111   23.08%   0.0  0.00%   

                           机审通过件  人审拒绝   人审拒绝率  风控通过件    风控通过率  客户取消  出库前风控强拒  \
型号             date                                                             
iPhone15       2024-11-14   10.0   1.0   5.26%    9.0   47.37%     5      1.0   
               2024-11-15   10.0   0.0   0.00%   10.0   58.82%     5      0.0   
iPhone15Plus   2024-11-15    2.0   0.0   0.00%    2.0  100.00%     0      0.0   
iPhone15Pro    2024-11-14    5.0   0.0   0.00%    5.0   55.56%     2      0.0   
               2024-11-15    6.0   1.0  10.00%    5.0   50.00%     1      0.0   
iPhone15ProMax 2024-11-14   12.0   1.0   1.96%   11.0   21.57%     2      0.0   
               2024-11-15   30.0   5.0   4.50%   25.0   22.52%     4      5.0   

                          出库前强拒比例  是否出库    进件出库率    订单出库率  
型号             date                                        
iPhone15       2024-11-14   5.26%     0    0.00%    0.00%  
               2024-11-15   0.00%     5   29.41%    6.58%  
iPhone15Plus   2024-11-15   0.00%     2  100.00%  100.00%  
iPhone15Pro    2024-11-14   0.00%     3   33.33%    6.25%  
               2024-11-15   0.00%     1   10.00%    2.00%  
iPhone15ProMax 2024-11-14   0.00%     9   17.65%    3.05%  
               2024-11-15   4.50%    13   11.71%    2.70%

In [ ]:
df_15_zm = df_15[df_15['归属渠道']=='芝麻租物']
df_15_zm2 = df_152[df_152['归属渠道']=='芝麻租物']
df_15_all_zm = group_data(df_15_zm,df_15_zm2,df_risk_examine, ['下单日期', 'hour'])
df_15_all_zm = df_15_all_zm[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例",'取消率', "是否出库","进件出库率","订单出库率"]]
df_15_all_zm

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,取消率,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,
2024-11-14,214,72,33.64%,38,17.76%,0.0,0.00%,18.0,0.0,0.00%,18.0,47.37%,6,1.0,2.63%,15.79%,8,21.05%,3.74%
2024-11-15,231,86,37.23%,52,22.51%,0.0,0.00%,25.0,4.0,7.69%,21.0,40.38%,5,2.0,3.85%,9.62%,11,21.15%,4.76%


In [ ]:
df_15_zm_all = group_data(df_15_zm,df_15_zm2,df_risk_examine,  ['型号', 'hour'])
df_15_zm_all = df_15_zm_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_15_zm_all

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
型号,,,,,,,,,,,,,,,,,,
iPhone15,127,35,27.56%,28,22.05%,0.0,0.00%,15.0,0.0,0.00%,15.0,53.57%,7,1.0,3.57%,4,14.29%,3.15%
iPhone15Plus,2,0,0.00%,2,100.00%,0.0,0.00%,2.0,0.0,0.00%,2.0,100.00%,0,0.0,0.00%,2,100.00%,100.00%
iPhone15Pro,56,19,33.93%,13,23.21%,0.0,0.00%,8.0,1.0,7.69%,7.0,53.85%,3,0.0,0.00%,2,15.38%,3.57%
iPhone15ProMax,206,79,38.35%,41,19.90%,0.0,0.00%,18.0,3.0,7.32%,15.0,36.59%,1,2.0,4.88%,11,26.83%,5.34%


In [ ]:
df_15_fzm = df_15[df_15['归属渠道']!='芝麻租物']
df_15_fzm2 = df_152[df_152['归属渠道']!='芝麻租物']
df_15_fzm_all = group_data(df_15_fzm,df_15_fzm2,df_risk_examine, ['型号', 'hour'])
df_15_fzm_all = df_15_fzm_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_15_fzm_all

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
型号,,,,,,,,,,,,,,,,,,
iPhone15,36,9,25.00%,10,27.78%,0.0,0.00%,5.0,1.0,10.00%,4.0,40.00%,3,0.0,0.00%,1,10.00%,2.78%
iPhone15Pro,28,6,21.43%,4,14.29%,0.0,0.00%,3.0,0.0,0.00%,3.0,75.00%,0,0.0,0.00%,2,50.00%,7.14%
iPhone15ProMax,671,129,19.23%,133,19.82%,0.0,0.00%,24.0,3.0,2.26%,21.0,15.79%,5,3.0,2.26%,11,8.27%,1.64%


In [ ]:
df_15_ss = df_15[df_15['归属渠道']=='搜索渠道']
df_15_ss2 = df_152[df_152['归属渠道']=='搜索渠道']
df_15_all_ss = group_data(df_15_ss,df_15_ss2,df_risk_examine, ['下单日期', 'hour'])
df_15_all_ss = df_15_all_ss[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例",'取消率', "是否出库","进件出库率","订单出库率"]]
df_15_all_ss

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,取消率,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,
2024-11-14,73,9,12.33%,15,20.55%,0.0,0.00%,5.0,1.0,6.67%,4.0,26.67%,1,0.0,0.00%,6.67%,3,20.00%,4.11%
2024-11-15,98,3,3.06%,30,30.61%,0.0,0.00%,8.0,0.0,0.00%,8.0,26.67%,2,0.0,0.00%,6.67%,5,16.67%,5.10%


In [ ]:
df_16_all = group_data(df_16,df_162,df_risk_examine, ['型号', 'hour'])
# df_16_all = group_data(df_16, '下单日期')
df_16_all = df_16_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_16_all

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
型号,,,,,,,,,,,,,,,,
iPhone16,240,53,22.08%,57,23.75%,18.0,31.58%,20.0,19.0,33.33%,7,1.0,1.75%,9,15.79%,3.75%
iPhone16Plus,57,12,21.05%,12,21.05%,1.0,8.33%,8.0,7.0,58.33%,1,3.0,25.00%,3,25.00%,5.26%
iPhone16Pro,722,200,27.70%,166,22.99%,26.0,15.66%,67.0,54.0,32.53%,13,10.0,6.02%,26,15.66%,3.60%
iPhone16ProMax,2522,639,25.34%,551,21.85%,135.0,24.50%,166.0,151.0,27.40%,42,15.0,2.72%,85,15.43%,3.37%


In [ ]:
df_16_date_group = group_data_new(df_16,df_162,df_risk_examine, ['型号', 'date', 'hour'])[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", "人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_16_date_group
# with pd.ExcelWriter('F:/需求/瑞彤需求/iPhone转化.xlsx', engine='openpyxl', mode='a') as writer:
#     df_16_date_group.to_excel(writer, sheet_name='iPhone16')

去重订单数  前置拦截     拦截率  是否进件  预授权通过率  机审强拒    强拒比例  \
型号             date                                                          
iPhone16       2024-11-14    106    31  29.25%    24  22.64%   4.0  16.67%   
               2024-11-15     69     9  13.04%    22  31.88%   6.0  27.27%   
iPhone16Plus   2024-11-14     37    10  27.03%     9  24.32%   0.0   0.00%   
               2024-11-15      4     0   0.00%     2  50.00%   0.0   0.00%   
iPhone16Pro    2024-11-14    303    82  27.06%    69  22.77%   5.0   7.25%   
               2024-11-15    290    80  27.59%    69  23.79%  14.0  20.29%   
iPhone16ProMax 2024-11-14   1101   267  24.25%   269  24.43%  53.0  19.70%   
               2024-11-15   1394   361  25.90%   279  20.01%  82.0  29.39%   

                           机审通过件  人审拒绝   人审拒绝率  风控通过件   风控通过率  客户取消  出库前风控强拒  \
型号             date                                                            
iPhone16       2024-11-14   11.0   0.0   0.00%   11.0  45.83%     5      1.0   
               2024-11-15    9.0   1.0   4.55%    8.0  36.36%     2      0.0   
iPhone16Plus   2024-11-14    7.0   1.0  11.11%    6.0  66.67%     1      3.0   
               2024-11-15    1.0   0.0   0.00%    1.0  50.00%     0      0.0   
iPhone16Pro    2024-11-14   37.0   7.0  10.14%   30.0  43.48%     7      6.0   
               2024-11-15   30.0   6.0   8.70%   24.0  34.78%     6      4.0   
iPhone16ProMax 2024-11-14   88.0   6.0   2.23%   82.0  30.48%    16     10.0   
               2024-11-15   78.0   9.0   3.23%   69.0  24.73%    26      5.0   

                          出库前强拒比例  是否出库   进件出库率   订单出库率  
型号             date                                      
iPhone16       2024-11-14   4.17%     4  16.67%   3.77%  
               2024-11-15   0.00%     5  22.73%   7.25%  
iPhone16Plus   2024-11-14  33.33%     2  22.22%   5.41%  
               2024-11-15   0.00%     1  50.00%  25.00%  
iPhone16Pro    2024-11-14   8.70%    13  18.84%   4.29%  
               2024-11-15   5.80%    13  18.84%   4.48%  
iPhone16ProMax 2024-11-14   3.72%    50  18.59%   4.54%  
               2024-11-15   1.79%    35  12.54%   2.51%

In [ ]:
df_16_zm = df_16[df_16['归属渠道']=='芝麻租物']
df_16_zm2 = df_162[df_162['归属渠道']=='芝麻租物']
df_16_all_zm = group_data(df_16_zm,df_16_zm2,df_risk_examine, ['下单日期', 'hour'])
df_16_all_zm = df_16_all_zm[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例",'取消率', "是否出库","进件出库率","订单出库率"]]
df_16_zm.shape
df_16_all_zm

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,取消率,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,
2024-11-14,814,271,33.29%,151,18.55%,3.0,1.99%,71.0,4.0,2.65%,67.0,44.37%,18,11.0,7.28%,11.92%,32,21.19%,3.93%
2024-11-15,736,222,30.16%,134,18.21%,4.0,2.99%,50.0,2.0,1.49%,48.0,35.82%,17,6.0,4.48%,12.69%,22,16.42%,2.99%


In [ ]:
df_16_zm_all = group_data(df_16_zm,df_16_zm2,df_risk_examine, ['型号', 'hour'])
df_16_zm_all = df_16_zm_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_16_zm_all

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
型号,,,,,,,,,,,,,,,,,,
iPhone16,47,16,34.04%,11,23.40%,1.0,9.09%,9.0,0.0,0.00%,9.0,81.82%,3,0.0,0.00%,4,36.36%,8.51%
iPhone16Plus,30,10,33.33%,7,23.33%,0.0,0.00%,6.0,1.0,14.29%,5.0,71.43%,1,2.0,28.57%,2,28.57%,6.67%
iPhone16Pro,355,115,32.39%,73,20.56%,1.0,1.37%,37.0,4.0,5.48%,33.0,45.21%,7,9.0,12.33%,13,17.81%,3.66%
iPhone16ProMax,979,302,30.85%,178,18.18%,2.0,1.12%,69.0,1.0,0.56%,68.0,38.20%,24,6.0,3.37%,35,19.66%,3.58%


In [ ]:
df_16_fzm = df_16[df_16['归属渠道']!='芝麻租物']
df_16_fzm2 = df_162[df_162['归属渠道']!='芝麻租物']
df_16_fzm_all = group_data(df_16_fzm,df_16_fzm2,df_risk_examine, ['型号', 'hour'])
df_16_fzm_all = df_16_fzm_all[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例","是否出库","进件出库率","订单出库率"]]
df_16_fzm_all

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,是否出库,进件出库率,订单出库率
型号,,,,,,,,,,,,,,,,,,
iPhone16,138,26,18.84%,34,24.64%,14.0,41.18%,11.0,1.0,2.94%,10.0,29.41%,4,1.0,2.94%,5,14.71%,3.62%
iPhone16Plus,14,0,0.00%,3,21.43%,0.0,0.00%,2.0,0.0,0.00%,2.0,66.67%,0,1.0,33.33%,1,33.33%,7.14%
iPhone16Pro,289,60,20.76%,77,26.64%,20.0,25.97%,30.0,9.0,11.69%,21.0,27.27%,6,1.0,1.30%,13,16.88%,4.50%
iPhone16ProMax,1534,334,21.77%,372,24.25%,133.0,35.75%,97.0,14.0,3.76%,83.0,22.31%,18,9.0,2.42%,50,13.44%,3.26%


In [ ]:
df_16_ss = df_16[df_16['归属渠道']=='搜索渠道']
df_16_ss2 = df_162[df_162['归属渠道']=='搜索渠道']
df_16_all_ss = group_data(df_16_ss,df_16_ss2,df_risk_examine, ['下单日期', 'hour'] )
df_16_all_ss = df_16_all_ss[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件","人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', "出库前风控强拒","出库前强拒比例",'取消率', "是否出库","进件出库率","订单出库率"]]
df_16_all_ss

,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,出库前风控强拒,出库前强拒比例,取消率,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,
2024-11-14,507,51,10.06%,153,30.18%,43.0,28.10%,48.0,4.0,2.61%,44.0,28.76%,7,3.0,1.96%,4.58%,30,19.61%,5.92%
2024-11-15,548,53,9.67%,166,30.29%,79.0,47.59%,42.0,8.0,4.82%,34.0,20.48%,9,1.0,0.60%,5.42%,23,13.86%,4.20%


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/iPhone转化数据_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl') as writer:
    df_15_all.to_excel(writer, sheet_name='iPhone15')
with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/iPhone转化数据_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl', mode='a') as writer:
    df_16_all.to_excel(writer, sheet_name='iPhone16')
    df_all_group.to_excel(writer, sheet_name='总体')

### 活动和非活动对比

In [ ]:
def qudao_types(a,b,c):
    a = str(a)
    b = str(b)
    if "租物iPhone16活动" in b:
        return "租物iPhone16活动"
    elif "芝麻信用租物夏促" in a:
        return "芝麻信用租物夏促"
    elif "租物618活动" in a:
        return "租物618活动"
    elif "租物超值租" in a:
        return "租物超值租"
    elif "租物五一活动" in a:
        return "租物五一活动"
    elif c == 1:
        return '芝麻租物'
    elif"租物" in b:
        return "芝麻租物"
    elif "芝麻" in b:
        return "芝麻租物"
    elif "租物" in a:
        return "芝麻租物"
    elif "芝麻" in a:
        return "芝麻租物"

df_16.loc[:,"芝麻租物渠道"]=df_16.apply(lambda x:qudao_types(x["来源渠道"],x["activity_name"],x["order_method"]),axis=1)
df_16["芝麻租物渠道"].value_counts()

芝麻租物渠道
芝麻租物            1626
租物iPhone16活动     124
芝麻信用租物夏促          16
租物618活动            7
Name: count, dtype: int64

In [ ]:
df_16 = df_16[~df_16['芝麻租物渠道'].isnull()]

In [ ]:
df_16_fzmzw = df_16[df_16['芝麻租物渠道']!='芝麻租物']
df_16_fzmzw2 = df_162[df_162['芝麻租物渠道']!='芝麻租物']
df_16_fzmzw_group = group_data(df_16_fzmzw,df_16_fzmzw2,df_risk_examine,  ['下单日期', 'hour'])

df_16_fzmzw_jj = df_16_fzmzw[df_16_fzmzw['是否进件']==1]
df_16_fzmzw_reason = df_16_fzmzw_jj[['下单日期', 'order_number', 'cancel_reason', 'rejected']]
df_16_fzmzw_reason.rename(columns={'order_number': '订单号', 'cancel_reason': '取消原因', 'rejected': '拒绝原因'}, inplace=True)

df_16_fzmzw_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", "人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', '取消率', '无法联系', '无法联系占比', "出库前风控强拒","出库前强拒比例", '待审核', "是否出库","进件出库率","订单出库率"]]


KeyError: '芝麻租物渠道'

In [ ]:
df_16_zmzw = df_16[df_16['芝麻租物渠道']=='芝麻租物']
df_16_zmzw_group = group_data(df_16_zmzw, '下单日期')

df_16_zmzw_jj = df_16_zmzw[df_16_zmzw['是否进件']==1]
df_16_zmzw_reason = df_16_zmzw_jj[['下单日期', 'order_number', 'cancel_reason', 'rejected']]
df_16_zmzw_reason.rename(columns={'order_number': '订单号', 'cancel_reason': '取消原因', 'rejected': '拒绝原因'}, inplace=True)

df_16_zmzw_group[["去重订单数","前置拦截","拦截率","是否进件","预授权通过率","机审强拒","强拒比例","机审通过件", "人审拒绝", '人审拒绝率', "风控通过件","风控通过率", '客户取消', '取消率', '无法联系', '无法联系占比', "出库前风控强拒","出库前强拒比例", '待审核', "是否出库","进件出库率","订单出库率"]]


,去重订单数,前置拦截,拦截率,是否进件,预授权通过率,机审强拒,强拒比例,机审通过件,人审拒绝,人审拒绝率,风控通过件,风控通过率,客户取消,取消率,无法联系,无法联系占比,出库前风控强拒,出库前强拒比例,待审核,是否出库,进件出库率,订单出库率
下单日期,,,,,,,,,,,,,,,,,,,,,,
2024-11-05,592,162,27.36%,124,20.95%,87,70.16%,37,1,0.81%,36,29.03%,13,10.48%,0,0.00%,3,2.42%,4,16,12.90%,2.70%


#### 导出取消原因

In [ ]:
# Today1 = str(datetime.now().strftime('%Y%m%d%H'))
# with pd.ExcelWriter(f'F:/需求/瑞彤需求/取消原因_{date1}-{date2}.xlsx', engine='openpyxl') as writer:
#     df_16_zmzw_reason.to_excel(writer, sheet_name='非四个活动取消原因', index=False)
#     df_16_fzmzw_reason.to_excel(writer, sheet_name='四个活动取消原因', index=False)

# iPhone16数据对比

### iPhone进件

In [ ]:
df_group_15_1 = iPhone_model(df_15, ['下单日期', '型号'])
df_group_15_1 = df_group_15_1.reset_index()
df_group_15_1 = df_group_15_1.pivot(index='下单日期', columns='型号', values='进件')
df_group_15_1

型号,iPhone15,iPhone15Plus,iPhone15Pro,iPhone15ProMax
下单日期,,,,
2024-11-05,34,1,24,124


In [ ]:
df_group_16_1 = iPhone_model(df_16, ['下单日期', '型号'])
df_group_16_1 = df_group_16_1.reset_index()
df_group_16_1 = df_group_16_1.pivot(index='下单日期', columns='型号', values='进件')
df_group_16_1

型号,iPhone16,iPhone16Plus,iPhone16Pro,iPhone16ProMax
下单日期,,,,
2024-11-05,7,6,24,106


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/iPhone进件数据_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl') as writer:
    df_group_15_1.to_excel(writer, sheet_name='iPhone15')
with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/iPhone进件数据_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl', mode='a') as writer:
    df_group_16_1.to_excel(writer, sheet_name='iPhone16')

### 华为系列

In [ ]:
df_hw = HuaWei(df,  date1, date2)
# df_hw['型号'].drop_duplicates()
df_group_hw = iPhone_model(df_hw, '型号')
# len_15 = len(df_group_15)
# ck_15 = df_group_15['出库'].sum()
# df_group_15
df_group_hw
# with pd.ExcelWriter(f'F:/myfile/业务相关数据/华为/华为转化数据/机型转化_{Today1}.xlsx', engine='openpyxl') as writer:
#     df_group_hw.to_excel(writer, sheet_name='华为机型转化')

,去重订单数,进件,进件占比,出库,出库占比,进件出库率
型号,,,,,,
华为Mate60Pro,1,0,0.00%,0,0.00%,nan%
华为Pura70,14,1,7.14%,0,0.00%,0.00%
华为Pura70Pro,10,3,30.00%,0,0.00%,0.00%
华为Pura70Pro+,4,4,100.00%,0,0.00%,0.00%
华为畅享70Pro,1,1,100.00%,0,0.00%,0.00%


# 出库

In [ ]:
def iPhone_ck(df, model_num, df_ck_all):
    df = df[df['型号'].str.contains(fr'iPhone {model_num}|iPhone{model_num}')]
    df['型号'] = df['型号'].str.replace(' ', '')
    df['型号'] = df['型号'].str.extract(fr'(iPhone ?{model_num}(ProMax|Pro|Plus)?)', expand=True)[0]
    df['型号内存'] = df['型号']+'_'+df['内存_y']


    df_groups = df.groupby('型号内存').agg({'是否出库': 'sum'})
    df_groups_iPhone = df.groupby('型号').agg({'是否出库': 'sum'})
    df_ck = df['是否出库'].sum()
    df_groups = df_groups.reset_index()
    df_groups_iPhone = df_groups_iPhone.reset_index()
    df_groups.loc[len(df_groups)] = [f'iPhone{model_num}_All', df_ck]
    df_groups_iPhone.loc[len(df_groups_iPhone)] = [f'iPhone{model_num}_All', df_ck]
    df_groups
    df_groups['总体出库']  = df_ck_all
    df_groups_iPhone['总体出库'] = df_ck_all

    df_groups['总体出库率'] = df_groups['是否出库']/df_ck_all * 100
    df_groups['总体出库率'] = df_groups['总体出库率'].apply(lambda x: f'{x: .2f}%')

    df_groups_iPhone['总体出库率'] = df_groups_iPhone['是否出库']/df_ck_all* 100
    df_groups_iPhone['总体出库率'] = df_groups_iPhone['总体出库率'].apply(lambda x: f'{x: .2f}%')


    return df_groups, df_groups_iPhone

### 总体出库

In [ ]:
df_ck_all = dfck['是否出库'].sum()
df_ck_all

np.int64(1697)

In [ ]:
# iPhone 15 系列
df_groups_15, df_groups_iPhone15 = iPhone_ck(dfck, 15, df_ck_all)
df_groups_iPhone15
# df_groups_15

,型号,是否出库,总体出库,总体出库率
0,iPhone15,78,1697,4.60%
1,iPhone15Plus,5,1697,0.29%
2,iPhone15Pro,116,1697,6.84%
3,iPhone15ProMax,738,1697,43.49%
4,iPhone15_All,937,1697,55.22%


In [ ]:
# iPhone 16 系列
df_groups_16, df_groups_iPhone16 = iPhone_ck(dfck, 16, df_ck_all)
df_groups_16
df_groups_iPhone16

,型号,是否出库,总体出库,总体出库率
0,iPhone16,38,1697,2.24%
1,iPhone16Plus,16,1697,0.94%
2,iPhone16Pro,127,1697,7.48%
3,iPhone16ProMax,487,1697,28.70%
4,iPhone16_All,668,1697,39.36%


### 芝麻渠道出库

In [ ]:
dfck_zm = dfck[dfck['归属渠道']=='芝麻租物']
df_ck_zm = dfck_zm['是否出库'].sum()
df_ck_zm

np.int64(767)

In [ ]:
# 15系列
dfck_groups_15, dfck_groups_iPhone15 = iPhone_ck(dfck_zm, 15, df_ck_zm)
dfck_groups_15
dfck_groups_iPhone15

,型号,是否出库,总体出库,总体出库率
0,iPhone15,27,767,3.52%
1,iPhone15Plus,1,767,0.13%
2,iPhone15Pro,87,767,11.34%
3,iPhone15ProMax,348,767,45.37%
4,iPhone15_All,463,767,60.37%


In [ ]:
# 16系列
dfck_groups_16, dfck_groups_iPhone16 = iPhone_ck(dfck_zm, 16, df_ck_zm)
dfck_groups_16
dfck_groups_iPhone16

,型号,是否出库,总体出库,总体出库率
0,iPhone16,12,767,1.56%
1,iPhone16Plus,9,767,1.17%
2,iPhone16Pro,44,767,5.74%
3,iPhone16ProMax,191,767,24.90%
4,iPhone16_All,256,767,33.38%


In [ ]:

dfck_groups_iPhone = pd.concat([dfck_groups_iPhone15, dfck_groups_iPhone16])
dfck_groups_iPhone

,型号,是否出库,总体出库,总体出库率
0,iPhone15,27,767,3.52%
1,iPhone15Plus,1,767,0.13%
2,iPhone15Pro,87,767,11.34%
3,iPhone15ProMax,348,767,45.37%
4,iPhone15_All,463,767,60.37%
0,iPhone16,12,767,1.56%
1,iPhone16Plus,9,767,1.17%
2,iPhone16Pro,44,767,5.74%
3,iPhone16ProMax,191,767,24.90%
4,iPhone16_All,256,767,33.38%


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/机型转化数据_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl') as writer:
    dfck_groups_iPhone.to_excel(writer, sheet_name='机型转化（汇总）', index=False)

In [ ]:
dfck_groups = pd.concat([dfck_groups_15, dfck_groups_16])
dfck_groups

,型号内存,是否出库,总体出库,总体出库率
0,iPhone15Plus_128G,1,767,0.13%
1,iPhone15ProMax_256G,347,767,45.24%
2,iPhone15ProMax_512G,1,767,0.13%
3,iPhone15Pro_128G,2,767,0.26%
4,iPhone15Pro_256G,85,767,11.08%
5,iPhone15_128G,13,767,1.69%
6,iPhone15_256G,14,767,1.83%
7,iPhone15_All,463,767,60.37%
0,iPhone16Plus_128GB,2,767,0.26%
1,iPhone16Plus_256GB,7,767,0.91%


In [ ]:
with pd.ExcelWriter(f'F:/myfile/业务相关数据/iPhone/机型转化数据_{month_date1}.{day_date1}-{month_date2}.{day_date2}.xlsx', engine='openpyxl', mode='a') as writer:
       dfck_groups.to_excel(writer, sheet_name='机型转化', index=False)

## 出库占比（iPhone15Pro|ProMax）

## 近7天

In [ ]:
now = pd.Timestamp.now() 
sevent_ago = now - timedelta(days=8)
sevent_ago
dfck_7day = dfck[dfck['下单日期']>=sevent_ago]


### 总体出库

In [ ]:
# iPhone 15
dfck_7day_ck = dfck_7day['是否出库'].sum()
iP15_7day, iP15_7day_all = iPhone_ck(dfck_7day, 15, dfck_7day_ck)
iP15_7day_all

,型号,是否出库,总体出库,总体出库率
0,iPhone15,13,403,3.23%
1,iPhone15Pro,28,403,6.95%
2,iPhone15ProMax,113,403,28.04%
3,iPhone15_All,154,403,38.21%


In [ ]:
# iPhone 16
iP16_7day, iP16_7day_all = iPhone_ck(dfck_7day, 16, dfck_7day_ck)
iP16_7day_all


,型号,是否出库,总体出库,总体出库率
0,iPhone16,13,403,3.23%
1,iPhone16Plus,7,403,1.74%
2,iPhone16Pro,40,403,9.93%
3,iPhone16ProMax,179,403,44.42%
4,iPhone16_All,239,403,59.31%


### 芝麻租物出库

In [ ]:
# iPhone 15
dfck_zm7 = dfck_7day[dfck_7day['归属渠道']=='芝麻租物']
dfck_zm7_ck = dfck_zm7['是否出库'].sum()
iP15_zm_7, iP15_zm_all_7 = iPhone_ck(dfck_zm7, 15, dfck_zm7_ck)
iP15_zm_all_7


,型号,是否出库,总体出库,总体出库率
0,iPhone15,5,186,2.69%
1,iPhone15Pro,22,186,11.83%
2,iPhone15ProMax,56,186,30.11%
3,iPhone15_All,83,186,44.62%


In [ ]:
# iPhone 16
iP16_zm_7, iP16_zm_all_7 = iPhone_ck(dfck_zm7, 16, dfck_zm7_ck)
iP16_zm_all_7

,型号,是否出库,总体出库,总体出库率
0,iPhone16,5,186,2.69%
1,iPhone16Plus,4,186,2.15%
2,iPhone16Pro,10,186,5.38%
3,iPhone16ProMax,81,186,43.55%
4,iPhone16_All,100,186,53.76%


## 近15天

In [ ]:
fifteen_ago = now - timedelta(days=16)
fifteen_ago
dfck_15day = dfck[dfck['下单日期']>=fifteen_ago]
dfck_15day_ck = dfck_15day['是否出库'].sum()
dfck_15day_ck

np.int64(849)

### 总体出库

In [ ]:
# iPhone 15
iP15_15day, iP15_15day_all = iPhone_ck(dfck_15day, 15, dfck_15day_ck)
iP15_15day_all

,型号,是否出库,总体出库,总体出库率
0,iPhone15,33,849,3.89%
1,iPhone15Pro,44,849,5.18%
2,iPhone15ProMax,271,849,31.92%
3,iPhone15_All,348,849,40.99%


In [ ]:
# iPhone 16
iP16_15day, iP16_15day_all = iPhone_ck(dfck_15day, 16, dfck_15day_ck)
iP16_15day_all

,型号,是否出库,总体出库,总体出库率
0,iPhone16,30,849,3.53%
1,iPhone16Plus,14,849,1.65%
2,iPhone16Pro,85,849,10.01%
3,iPhone16ProMax,346,849,40.75%
4,iPhone16_All,475,849,55.95%


### 芝麻租物出库

In [ ]:
# iPhone 15
dfck_zm15 = dfck_15day[dfck_15day['归属渠道']=='芝麻租物']
iP15_zm_15, iP15_zm_all_15 = iPhone_ck(dfck_zm15,15, dfck_15day_ck)
iP15_zm_all_15

,型号,是否出库,总体出库,总体出库率
0,iPhone15,14,849,1.65%
1,iPhone15Pro,34,849,4.00%
2,iPhone15ProMax,118,849,13.90%
3,iPhone15_All,166,849,19.55%


In [ ]:
# iPhone 16
iP15_zm_16, iP15_zm_all_16 = iPhone_ck(dfck_zm15, 16, dfck_15day_ck)
iP15_zm_all_16

,型号,是否出库,总体出库,总体出库率
0,iPhone16,10,849,1.18%
1,iPhone16Plus,8,849,0.94%
2,iPhone16Pro,26,849,3.06%
3,iPhone16ProMax,132,849,15.55%
4,iPhone16_All,176,849,20.73%


In [ ]:
# with pd.ExcelWriter('F:/myfile/业务相关数据/iPhone/近7天出库占比.xlsx', engine='openpyxl') as writer:
#     iP15_qd_all.to_excel(writer, sheet_name='iPhon15(总体汇总)', index=False)
# with pd.ExcelWriter('F:/myfile/业务相关数据/iPhone/近7天出库占比.xlsx', engine='openpyxl', mode='a') as writer:
#     iP15_zm_all.to_excel(writer, sheet_name='iPhon15(芝麻租物汇总)', index=False)
#     iP15_qd.to_excel(writer, sheet_name='iPhon15(总体)', index=False)
#     iP15_zm.to_excel(writer, sheet_name='iPhon15(芝麻租物)', index=False)

In [ ]:
# with pd.ExcelWriter('F:/myfile/业务相关数据/iPhone/近15天出库占比.xlsx', engine='openpyxl') as writer:
#     iP15_qd_all_15.to_excel(writer, sheet_name='iPhon15(总体汇总)', index=False)
# with pd.ExcelWriter('F:/myfile/业务相关数据/iPhone/近15天出库占比.xlsx', engine='openpyxl', mode='a') as writer:
#     iP15_zm_all_15.to_excel(writer, sheet_name='iPhon15(芝麻租物汇总)', index=False)
#     iP15_qd_15.to_excel(writer, sheet_name='iPhon15(总体)', index=False)
#     iP15_zm_15.to_excel(writer, sheet_name='iPhon15(芝麻租物)', index=False)

In [ ]:
# with pd.ExcelWriter('F:/myfile/业务相关数据/iPhone/iPhone16出库占比.xlsx', engine='openpyxl') as writer:
#     iPhone16_all.to_excel(writer, sheet_name='iPhon16(总体汇总)', index=False)
# with pd.ExcelWriter('F:/myfile/业务相关数据/iPhone/iPhone16出库占比.xlsx', engine='openpyxl', mode='a') as writer:
#     iPhone16.to_excel(writer, sheet_name='iPhon16(含内存)', index=False)

# 设置定时任务

In [ ]:
# from apscheduler.schedulers.blocking import BlockingScheduler
# from apscheduler.schedulers.background import BackgroundScheduler

# # hour = 18
# # minute = 20
# H = datetime.now().strftime('%H')
# M = datetime.now().strftime('%M')
# def my_job1():
#     # print(f'执行定时任务：现在是每日的{hour}点{minute}分')
#     print(f'执行定时任务：现在是每日的{H}点{M}分')
#     path1 = r'\\digua\迪瓜租机\18.iPhone16预租数据/'
#     Today1 = str(datetime.now().strftime('%Y%m%d%H'))
#     with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl') as writer:
#         df_pivot_qcdds.to_excel(writer, sheet_name='iPhone16系列去重订单数汇总', index=False)
#     with pd.ExcelWriter(path1 + f'iPhone 16系列_{Today1}.xlsx', engine='openpyxl', mode='a') as writer:
#         df_pivot_jjs.to_excel(writer, sheet_name='iPhone16系列进件汇总', index=False)
#         qd_group.to_excel(writer, sheet_name='iPhone16系列明细')
# scheduler1 = BackgroundScheduler()
# # 添加一个cron任务，每天的16点55分执行
# # scheduler1.add_job(my_job1, 'cron', hour=hour, minute=minute)
# scheduler1.add_job(my_job1, 'cron', hour=H, minute=int(M)+1)

# # 启动scheduler1
# scheduler1.start()
    

In [ ]:
# df_16_group_qcdd = df_16_group.groupby(['归属渠道', '下单日期']).agg({'去重订单数': 'sum'}).reset_index()
# df_16_group_qcdd = df_16_group_qcdd.sort_values(by=['归属渠道', '下单日期'], ascending=(False, False))
# df_pivot_qcdd = df_16_group_qcdd.pivot(index='下单日期', columns='归属渠道', values='去重订单数')
# df_pivot_qcdd = df_pivot_qcdd[['付费灯火','八派信息','单人聊天会话中的小程序消息卡片（分享）','我的小程序入口','搜索渠道','支付宝社群','生活号','芝麻租物']]
# df_pivot_qcdd

# 待审核

In [ ]:
# df_dsh = df[df['待审核']==1]
# df_dsh = df_dsh[df_dsh['product_name'].str.contains(r'iPhone 16|iPhone16')]
# df_dsh = df_dsh[['order_number', 'create_time', 'product_name', '租赁方案', '内存', '颜色', '来源渠道']]
# df_dsh.rename(columns={'order_number': '订单号', 'create_time': '下单时间', 'product_name': '型号'}, inplace=True)
# df_dsh


# with pd.ExcelWriter('F:/需求/iPhone16.xlsx', engine='openpyxl') as writer:
#     df_dsh.to_excel(writer, sheet_name='待审核', index=False)